## Preprocessing

In [53]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Note:
1. The variable that is the target for the model is "IS_SUCCESSFUL"
2. Currently, all other variables are features for the model, e.g. "APPLICATION_TYPE", "AFFILIATION", etc.

In [54]:
#examine the data - determine how many unique values exist within each column
unique_counts = application_df.nunique()
print(unique_counts)

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [55]:
# Drop the non-beneficial ID column: 'EIN'.
application_df.drop(["EIN"], axis=1,inplace=True)
application_df.head()


,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [56]:
#verify the EIN column was dropped

unique_counts = application_df.nunique()
print(unique_counts)

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [57]:
# Based on the initial model I built, I identifed two columns whose values can be binned:
# APPLICATION_TYPE
# CLASSIFICATION


In [58]:
#In an attempt to improve the model, I'll look at the "NAME" column. It's possible there are numerous
#records for each unique NAME
value_counts_name = application_df["NAME"].value_counts()
print(value_counts_name)

NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: count, Length: 19568, dtype: int64


In [59]:
# Look at NAME value counts >100
value_counts_name_greater_than_1 = value_counts_name[value_counts_name>100]
print(value_counts_name_greater_than_1)

NAME
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCI

Note: There are 19,568 unique names in the data.
* 19,538 names appear fewer than 100 times
* 30 names appear at least 100 times

Of note, Parent Booster USA Inc. appears more than 1,000 times, and "Tops Club Inc" appears 765 times.

Given the frequency of names, it's worth trying to group (aka, bin) the names whose frequency is less than an arbitrary number. I'll try 100.

In [60]:
#*************************
# binning, part 01 of 03**
#*************************

application_types_to_replace = ['T9','T13','T12','T2','T14','T25','T15','T29','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [61]:
#*************************
# binning, part 02 of 03**
#*************************

values_to_replace = application_df["CLASSIFICATION"].value_counts().index[-66:]
application_df["CLASSIFICATION"]=application_df["CLASSIFICATION"].replace(values_to_replace,"Other")
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [62]:
#*************************
# binning, part 03 of 03**
#*************************

values_to_replace = application_df["NAME"].value_counts().index[-19538:]
application_df["NAME"]=application_df["NAME"].replace(values_to_replace,"Other")
application_df['NAME'].value_counts()

NAME
Other                                                                 25987
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CL

In [63]:
# Convert categorical data to numeric with `pd.get_dummies`
copied_df= application_df.copy()  #copy the DataFrame so if my approach doesn't work, I don't have to start from the beginning.
application_df_new = pd.get_dummies(application_df)
application_df_new.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,NAME_FARMERS EDUCATIONAL AND COOPERATIVE UNION OF AMERICA,NAME_HABITAT FOR HUMANITY INTERNATIONAL INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [64]:
#use the following code to confirm the features that are used in the model
unique_counts_new = application_df_new.nunique()
print(unique_counts_new)

STATUS                                                     2
ASK_AMT                                                 8747
IS_SUCCESSFUL                                              2
NAME_ALPHA PHI SIGMA                                       2
NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC       2
                                                        ... 
INCOME_AMT_25000-99999                                     2
INCOME_AMT_50M+                                            2
INCOME_AMT_5M-10M                                          2
SPECIAL_CONSIDERATIONS_N                                   2
SPECIAL_CONSIDERATIONS_Y                                   2
Length: 75, dtype: int64


In [65]:
# Split the preprocessed data into features and target arrays
y=application_df_new['IS_SUCCESSFUL']
X=application_df_new.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [66]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [73]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=74))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                6000      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8461 (33.05 KB)
Trainable params: 8461 (33.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [74]:
#create a model checkpoint callback to save the model's weights every 5 epochs


# Define the filepath where the model weights will be saved
filepath = "model_weights_optimization.h5"

# Create a ModelCheckpoint callback to save the model weights every 5 epochs
checkpoint_callback = ModelCheckpoint(filepath,
                                      save_weights_only=True,
                                      save_freq='epoch',
                                      period=5)


In [75]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [76]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5142 - accuracy: 0.7397
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4941 - accuracy: 0.7558
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4918 - accuracy: 0.7579
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4896 - accuracy: 0.7572
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4879 - accuracy: 0.7592
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4865 - accuracy: 0.7591
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4864 - accuracy: 0.7586
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4849 - accuracy: 0.7600
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4838 - accuracy: 0.7600
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4837 - accura

In [77]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5068 - accuracy: 0.7523 - 446ms/epoch - 2ms/step
Loss: 0.5068155527114868, Accuracy: 0.7523031830787659


In [78]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn_model.save('AphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [79]:
from google.colab import files
files.download('AphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>